In [1]:
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import pytesseract
import re
import math
import tensorflow as tf


busNum = 2233
busplainNum = 2749
global busNum, busplainNum

store_img = ['./result/bus_num_fr/bus_num_fr.jpg', './result/bus_num_side/bus_num_side.jpg', './result/license_plate/license_plate.jpg']

def OCR(img):
    ## Original Image Load
    box = cv2.imread(img)
    
    ## 배경 Gray로 설정
    gray = cv2.cvtColor(box, cv2.COLOR_RGB2GRAY)

    ## 이미지 사이즈 정규화(확대)
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)

    ## 이미지 블러처리하기
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    ## 이미지 흑백 대조하기, 최적 임계값을 자동으로 추출하는 Otsus 사용
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

    ## 확장을 위한 커널 생성
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

    ## 글씨 이미지 크기 확장
    dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

    # find contours of regions of interest within license plate
    try:
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    except:
        ret_img, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # sort contours left-to-right
    sorted_contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

    # create copy of gray image
    im2 = gray.copy()

    # create blank string to hold license plate number
    plate_num = ""

    # loop through contours and find individual letters and numbers in license plate
    for cnt in sorted_contours:
        x,y,w,h = cv2.boundingRect(cnt)
        height, width = im2.shape
        # if height of box is not tall enough relative to total height then skip
        if height / float(h) > 6: continue

        ratio = h / float(w)
        # if height to width ratio is less than 1.5 skip
        if ratio < 1.5: continue

        # if width is not wide enough relative to total width then skip
        if width / float(w) > 15: continue

        area = h * w
        # if area is less than 100 pixels skip
        if area < 100: continue

        # draw the rectangle
        rect = cv2.rectangle(im2, (x,y), (x+w, y+h), (0,255,0),2)

        # grab character region of image
        roi = thresh[y-5:y+h+5, x-5:x+w+5]

        # perfrom bitwise not to flip image to black text on white background
        roi = cv2.bitwise_not(roi)

        # perform another blur on character region
        roi = cv2.medianBlur(roi, 5)

        try:
            text = pytesseract.image_to_string(roi, config='-c tessedit_char_whitelist=0123456789 --psm 8 --oem 3')
            # clean tesseract text by removing any unwanted blank spaces
            clean_text = re.sub('[\W_]+', '', text)
            plate_num += clean_text
        except: 
            text = None
    if plate_num != None:
        return plate_num
    else:
        return False
    
    # 노이즈 제거 해보기    
            

def tfBusNum(store_img):
    for img in store_img:
        try:
            preBusNum = str(OCR(img)).strip()
            print(f'busNum: {busNum}')
            print(f'prediction busNum: {preBusNum}')
            if preBusNum == str(busNum) or preBusNum == str(busplainNum):
                return True  
        except:
            continue
    else:
        return False
tfBusNum(store_img)

busNum: 2233
prediction busNum: 2233


True